In [144]:
import pandas as pd
import requests
import calendar

## HORAS SOLARES ANUALES

dfsolar = pd.read_csv('datos_solares.csv').drop_duplicates().drop(0).reset_index(drop=True)
dfsolar.columns = ['Ciudad','Horas de sol','Irradiación']
dfsolar['Horas de sol'] = dfsolar['Horas de sol'].astype(float)*1000

#Irradiación solar [kWh/m2·día]
# Convierte los valores de la columna a float
dfsolar['Irradiación'] = dfsolar['Irradiación'].str.replace(' kWh/m2día', '')
dfsolar['Irradiación'] = dfsolar['Irradiación'].str.replace(',', '.')

### DATOS HORARIOS POR CIUDAD

# URL del archivo de texto
def sacar_datos_horarios(ciudad):    
    url = f'https://cdn.mitma.gob.es/portal-web-drupal/salidapuestasol/2024/{ciudad}-2024.txt'

    # Descargar contenido
    response = requests.get(url)
    response.encoding = 'ISO-8859-1'
    content = response.text

    # Dividir en líneas y omitir las filas de encabezado y pie
    lines = content.splitlines()[4:6] + content.splitlines()[7:]  # Quitar encabezado

    # Crear listas de listas para estructurar datos manualmente,
    data = []

    for line in lines:
        # Trata de dividir las líneas de manera personalizada
        parts = line.split() 
        data.append(parts)

    # Ajustamos para los meses que no tienen ni 30 ni 31 dias

    def adjust_days(data):
    # Recorrer cada sublista en la lista de datos
        for sublist in data:
            # Verificar si la sublista corresponde al día 30
            if not sublist:
                continue
            elif sublist[0] == '30':
            # Corregimos Febrero (posiciones 3 y 4)
                sublist.insert(3, '0000')
                sublist.insert(4, '0000')
            elif sublist[0] == '31':
            # Corregimos Febrero, Abril, Junio, Septiembre y Noviembre
                sublist.insert(3, '0000')
                sublist.insert(4, '0000')
                sublist.insert(7, '0000')
                sublist.insert(8, '0000')
                sublist.insert(11, '0000')
                sublist.insert(12, '0000')
                sublist.insert(17, '0000')
                sublist.insert(18, '0000')
                sublist.insert(21, '0000')
                sublist.insert(22, '0000')
            
        return data
    
    data = adjust_days(data)

    # Columnas de cabeceras
    columnas = ['Dia', 'Ene_Ort', 'Ene_Ocas', 'Feb_Ort', 'Feb_Ocas', 'Mar_Ort', 'Mar_Ocas', 'Apr_Ort', 'Apr_Ocas', 
                'May_Ort', 'May_Ocas', 'Jun_Ort', 'Jun_Ocas', 'Jul_Ort', 'Jul_Ocas', 'Aug_Ort', 'Aug_Ocas',
            'Sep_Ort', 'Sep_Ocas', 'Oct_Ort', 'Oct_Ocas', 'Nov_Ort', 'Nov_Ocas', 'Dec_Ort', 'Dec_Ocas']

    # Crear DataFrame
    df = pd.DataFrame(data).iloc[:-4].dropna(axis=1, how='all')
    df.columns = columnas
    df = df.iloc[2:].reset_index(drop=True)

    def format_time(value):
        if isinstance(value, str):
            # Asegura que los valores tengan cuatro caracteres
            if len(value) == 3:
                value = '0' + value
            # Inserta ':' antes del penúltimo carácter
            value = value[:-2] + ':' + value[-2:]
        return value

    # Aplica la función a todas las columnas excepto la primera
    df.iloc[:, 1:] = df.iloc[:, 1:].applymap(format_time)

    # Vamos a transformar los datos en formato hora

    for col in df.columns[1:]:  # Ignora la primera columna ('Dia')
        # Convertir a Timedelta
        df[col] = pd.to_timedelta(df[col] + ':00')

    # Lista de meses abreviados
    meses = ["Ene", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
    # Crear el nuevo DataFrame con la columna 'Dia'
    nuevo_df = pd.DataFrame()
    nuevo_df['Dia'] = df['Dia']
    nuevo_df['Ciudad'] = ciudad

    # Calcular la duración de cada mes
    for mes in meses:
        nuevo_df[mes] = (df[f'{mes}_Ocas'] - df[f'{mes}_Ort']).apply(lambda x: x.total_seconds() / 3600)

    return nuevo_df

df_ciudad = sacar_datos_horarios('Sevilla')
df_ciudad

## CÁLCULO PLACAS SOLARES

factor_solar = 0.8
pot_placa = 0.455 #kWp
dias = {
'Ene': 31,
'Feb': 29,
'Mar': 31,
'Apr': 30,
'May': 31,
'Jun': 30,
'Jul': 31,
'Aug': 31,
'Sep': 30,
'Oct': 31,
'Nov': 30,
'Dec': 31}

# Crear el diccionario para almacenar las sumas mensuales
meses = df_ciudad.columns.difference(['Dia', 'Ciudad'])
horas_sol = {mes: df_ciudad[mes].sum()*factor_solar for mes in meses}
porcen_sol = {mes: round(df_ciudad[mes].sum()*factor_solar/(dias[mes]*24),2) for mes in meses}
# Aproximadamente entre el 40% y el 50% de luz al día. Se toma 40% para ser conservativo
# Calcular el total de todos los meses

# Para un mes dado
mes = 'Jul'
c_punta = 188.39
c_llano = 233.13
c_valle = 355.02

#mes = 'Feb'
#c_punta = 188.39
#c_llano = 233.13
#c_valle = 355.02

c_total = c_punta + c_llano + c_valle

# El consumo anual de energía por parte del consumidor debe ser igual o mayor 
# al 80 %  de la energía anual generada por la instalación.

#c_total >= e_generada *0.8


n_placas = int(c_total / (pot_placa * 0.7 * horas_sol[mes] * 0.8))

e_generada = n_placas * pot_placa * 0.7 * horas_sol[mes]
e_generada #912.0481066666669

c_dia = c_total * 0.4
c_noche = c_total - c_dia

## AHORRO PLACAS

#Horas de Valle en un año: 4584
#Horas de Llano en un año: 2088
#Horas de Punta en un año: 2088

def calcular_horas_por_mes(month, year):
    # Definimos los períodos para un día laborable
    discriminacion = {
        "00": 'valle', "01": 'valle', "02": 'valle', "03": 'valle', 
        "04": 'valle', "05": 'valle', "06": 'valle', "07": 'valle',
        "08": 'llano', "09": 'llano', "10": 'punta', "11": 'punta',
        "12": 'punta', "13": 'punta', "14": 'llano', "15": 'llano',
        "16": 'llano', "17": 'llano', "18": 'punta', "19": 'punta',
        "20": 'punta', "21": 'punta', "22": 'llano', "23": 'llano',
    }

    # Calcula horas diarias por tipo para días laborables
    horas_valle_lab = sum(1 for h in discriminacion if discriminacion[h] == 'valle')
    horas_llano_lab = sum(1 for h in discriminacion if discriminacion[h] == 'llano')
    horas_punta_lab = sum(1 for h in discriminacion if discriminacion[h] == 'punta')

    # Obtén el número total de días en el mes
    num_dias_mes = calendar.monthrange(year, month)[1]

    # Contar cuántos sábados y domingos hay en ese mes
    dias_finde = sum(1 for d in range(1, num_dias_mes + 1) if calendar.weekday(year, month, d) >= 5)
    dias_laborables = num_dias_mes - dias_finde

    # Calcular horas totales para ese mes
    horas_valle_totales = (horas_valle_lab * dias_laborables) + (24 * dias_finde)
    horas_llano_totales = horas_llano_lab * dias_laborables
    horas_punta_totales = horas_punta_lab * dias_laborables

    return {
        'Valle': horas_valle_totales,
        'Llano': horas_llano_totales,
        'Punta': horas_punta_totales
    }

# Ejemplo de uso para marzo (mes 3) del año 2023
resultado = calcular_horas_por_mes(7, 2024)
print(resultado)


{'Valle': 376, 'Llano': 184, 'Punta': 184}
